#Token Ring

##Постановка задачи

Задача: Построение простой модели сетевого протокола TokenRing и исследовании его свойств

Этапы:
+ Реализовать модель протокола
+ Исследовать пропускную способность сети (throughput) и характерное время задержки (latency) в зависимости от количества узлов N и количества пакетов P
+ Проанализировать полученные результаты
+ Оптимизировать throughput или latency и исследовать влияние оптимизаций для одного режима на весь спектр режимов.


## Определения

+ Характерное время задержки (latency) - промежуток времени с момента отправки токена с ноды-отправителя до момента получения на ноде-получателе. 

+ Пропускную способность сети (throughput) - количество токенов прошедших через ноду за единицу времени.

+ Максимальное количество токенов в системе - размер очереди * количество нод (нод коннекторов), то есть совокупный размер очередей.



## Реализация

Протокол реализован на взаимодействии Node через NodeConnector, представляющий из себя блокирующую очередь (java.util.concurrent).
Ноды передают далее по кругу пакет - токен, содержащий id и вспомогательное поле за замера времени задержки.

При получении токена - нода его обрабатывает и передаёт дальше, не выбрасывая из системы.

Под обработкой подразумевается получение экспериментальных данных и выполение операций с токеном, в нашем случае никаких дополнительных операций не производится.

Ноды запускаются в разных потоках.


## Описание эспериментальной установки

Ноутбук MacBook Pro (2019):
+ 2,6 GHz 6‑ядерный процессор Intel Core i7
+ 16 ГБ 2667 MHz DDR4
+ macOS Catalina 10.15.7

## Постановка эксперимента

Эксперимент проводится с помощью библиотеки для модульного тестирования Junit. 

Предварительно проводится "прогрев кода" - работа протокола на 2х нода с 2мя токенами в течение 5 секунд.

Для latency - единицы измерения nanoseconds. Время замеряется с помощью System.nanotime().

Для throughput - единицы измерения количество токенов прошедших за 1 секунду.

Как сказано выше, для замера latency и throughput используется все ноды, соответственно, 

latency - это время, за которое определенный токен проходит целый круг ( нода считывает значения только для токенов с определенным id (id % кол-во нод == id ноды)) 

throughput - это общее количество токенов прошедших через ноду за 1 секунду. 

Таким образом мы можем оценить общие характеристики всех нод, системы в целом

## Первичный анализ системы

Проведём эксперимент по определению количества нод, доступных для проведения эксперимента - выберем количество потоков, которое мы можем использовать в эксперименте для общей наглядности тенденций.

Для этого запустим протокол на разном количестве нод.

![](https://drive.google.com/uc?export=view&id=1UicW1iAebl4shQuGaV1Q4InLy1PRAYHC) ![](https://drive.google.com/uc?export=view&id=1-15dfh6GF_g7HYqU3S2J81DM9T9mrITa)

Можно заметить, что latency растет с увеличением количества нод, что происходит из-за увеличения размеров круга, который нужно пройти токену для замера

throughput так же растёт вначале и выходит на планку в конце, с необычной просадкой в середине.



## Определение зависимости latency и throughput от размера очереди.

Далее представлены зависимости latency и throughput от размера очереди при разном количестве токенов в системе (нагрузке на систему).

Размер очереди варируется от 2 до 8 так как далее выходит на планку.
Рассматриваем три варианта нагрузки.

Low load - количество токенов равно половине количества нод.

Medium load - количество токенов равно количестве нод.

High load - количество токенов равно 0.8 от 2 * количество нод.
То есть больше количества нод и меньше макисмального количества, так же подходит для любого количества нод с очередью размера 2 и более.

![](https://drive.google.com/uc?export=view&id=1-AU2fKifWCrDg0F4Cp0bxrOHYUlCy5Hk)![](https://drive.google.com/uc?export=view&id=1-Ax3OB8aC6F3cIJwLYl3WMcHXt9ruNUd)
![](https://drive.google.com/uc?export=view&id=1-G-DadjiW8_Cl5ENVBQ7CHAB6ZFRaO8c)![](https://drive.google.com/uc?export=view&id=106dsZpCQ5p5DnGR2bPrZth1JVp37SpVx)
![](https://drive.google.com/uc?export=view&id=1-JfaSONXwHK2isjiHYPTCnaKesrZ_kDK)![](https://drive.google.com/uc?export=view&id=105csgCqNny_mhxtQoRcMYpptVG8A9Xo9)

С увеличение количества нод увеличивается latency, что теоретически объяснимо - для прохождения полного круга необходимо пройти больше нод.

На первых двух графиках почти нет заметных изменений в latency, увеличение очереди почти не даёт ускорения(для кол-ва нод до 10), так как количество токенов меньше количества нод, и все токены успевают обработаться без ожидания в очереди.

Далее же можно наблюдать ускорение даже при малой нагрузке

При medium и high load отчетливо видно, что увеличение размера очереди вносит существенный вклад в ускорение

![](https://drive.google.com/uc?export=view&id=1-P5BRDFpM4c0cPQXBCcmkdb5zoAJtqy3)![](https://drive.google.com/uc?export=view&id=1-OY-7iek_-8Lm5Ko_IchZUsg9PpPrgyJ)
![](https://drive.google.com/uc?export=view&id=1-aJPjzlEYBMo4mhQZxiIuojDQLP7AdNq)![](https://drive.google.com/uc?export=view&id=1-aNd2GYeI9LIiqHWqnHvVlPjXIu2e8Gu)
![](https://drive.google.com/uc?export=view&id=103lS-DjfzabnYH2TPCCuqVxex4XORV9i)![](https://drive.google.com/uc?export=view&id=103NIc3plqyNDB1NDtVWB00YLdICaEhNA)

Аналогичную картину мы можем наблюдать и с throughput.
Изменения появляются в основном при средней и повышенной нагрузке и при большом кол-ве нод в системе (от 9), всё по тем же причинам.

Важно заметить, что обработка токенов не занимает много времени, поэтому мы получаем такие результаты на малой и средней нагрузках.

Далее следует более детально изучить зависимость latency и throughput от количества токенов в системе при разных размерах очереди.

## Определение зависимости latency и throughput от размера очереди (анализ влияния количества покетов в систмеме на показатели)

Для этого рассмотрим систему с 4, 8 и 12 нодами.
Проведём эксперимент для размеров очереди 4, 8, 12, 20, 28 с количеством токенов от 3 до максимального с шагом 4.

#### 4 ноды

![](https://drive.google.com/uc?export=view&id=106r6cGmEfwsVoe9hN9y6ZXq591S24aFX)
![](https://drive.google.com/uc?export=view&id=108wfxZfVidsuwKfA8WTmS8YtEkhLfmbq)

#### 8 нод

![](https://drive.google.com/uc?export=view&id=10JSn_81ImOhH6xMNxpz2VZnAZssZ6VVh)
![](https://drive.google.com/uc?export=view&id=10DzQS7JF8LZUqhrfXqBSZAY1OTPbSg3F)

#### 12 нод

![](https://drive.google.com/uc?export=view&id=10LypkAqu2T4WTv9WqLb67TWqCMHIwDLt)
![](https://drive.google.com/uc?export=view&id=10NSS5U0XHI9q7yYpTBT2xqyerkbS2UgZ)

По графикам видно, что предположение о эффективности увеличения размера очереди подтвердилось и в этом эксперименте. 
Для одинакового кол-ва токенов latency при большем размере очереди в разы меньше.

Аналогично и для throughput.

Так важно заметить существование оптимального количества токенов в системе для throughput. Которое примерно равно половине максимальной нагрузке, что соответствует заполнености очередей наполовину в среднем. Из-за чего, вероятнее всего, и получается максимум - две соседние ноды смогут обработать весь пулл токенов без торможения остальных.

Характер зависимостей остается таким же для всех вариантов системы (для 4, 8, 12 нод).

И при увеличении с 4 до 8 нод видно удлинение и оптимального участка для throughput, но при 12 нодах заметны симметричный впадины и появляется более явный оптимум


## Вывод

Задача: Построение простой модели сетевого протокола TokenRing и исследовании его свойств

Этапы:
В данной работе реализована модель протокола, исследованы throughput и latency в зависимости от количества нод и количества токенов, проанализированы полученные результаты

В результате, получены выводы, что latency прямо пропорциональнно зависит от размера очередей. Благодаря уменьшению времени ожидания нод на передачу токена следующей ноде.

Так же выявлено существование некого оптимального диапозона количества токенов в системе в зависимости от размера очередей, равного половине макисмального количества токенов. Это может быть практически использовано в определении размера очередей для конкретной системы, с известными средними нагрузками в ней.